In [1]:
from urllib.request import *
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import scipy.stats as stats
import pylab as pl
import requests
import json
import os
import time
%matplotlib inline

In [5]:
with open("../scraper/data/players_ref.json", "r") as in_file:
    players_ref_list = json.load(in_file)

In [6]:
len(players_ref_list)

12075

In [19]:
def parsePlayer(player_ref):    
    playerID = player_ref.split("/")[-1]
    
    with open("../scraper/data/players/" + playerID + "/page.html") as in_file:
        player_page = json.load(in_file)
    
    response = BeautifulSoup(player_page, 'html.parser')
    
    playerInfos = str(response.find("table", {"class":"auflistung"}))
    player = {}
    player["href"] = player_ref
    try:
        player["number"] = response.find("span", {"class":"dataRN"}).text
    except:
        player["number"] = None
    player["name"] = response.find("h1", {"itemprop":"name"}).text
    player["player_id"] = player_ref.split("/")[-1]
    position = BeautifulSoup(playerInfos.split("Position")[1], 'html.parser').find("td").text
    reg = re.compile( "[a-zA-Z -]")
    player["position"] = "".join(reg.findall(position))
    player["birthdate"] = BeautifulSoup(playerInfos.split("Date of birth")[1], 'html.parser').find("td").text
    player["nationality"] = BeautifulSoup(playerInfos.split("Nationality")[1], 'html.parser').find("td").find("img")["title"]
    player["current_club"] = BeautifulSoup(playerInfos.split("Current club")[1], 'html.parser').find("td").find_all("a")[-1].text

    try:
        transfers = []
        trans = response.find("div",{"class" : "box transferhistorie"}).find("table").find("tbody").find_all("tr", {"class":"zeile-transfer"})

        for t in trans:
            transfer = {}
            transfer["player"] = player_ref.split("/")[-1]
            transfer["date"] = t.find_all("td", {"class":"zentriert hide-for-small"})[1].text
            transfer["from"] = t.find_all("td", {"class":"no-border-rechts vereinswappen"})[0].find("a")["id"]
            transfer["to"] = t.find_all("td", {"class":"no-border-rechts vereinswappen"})[1].find("a")["id"]
            if (t.find("td", {"class":"zelle-abloese"}).text) == "End of loan" or t.find("td", {"class":"zelle-abloese"}).text =="Loan":
                transfer["fee"] = t.find("td", {"class":"zelle-mw"}).text
            else:
                transfer["fee"] = t.find("td",{"class":"zelle-abloese"}).text
            transfers.append(transfer)
    except:
        transfers = None
        
    return player, transfers

In [20]:
test = parsePlayer(players_ref_list[0])
test

({'birthdate': 'May 11, 1992 ',
  'current_club': 'Chelsea FC',
  'href': '/thibaut-courtois/profil/spieler/108390',
  'name': 'Thibaut Courtois',
  'nationality': 'Belgium',
  'number': '#13',
  'player_id': '108390',
  'position': 'Goalkeeper'},
 [{'date': 'Jun 30, 2014',
   'fee': '25,00 Mill. €',
   'from': '13',
   'player': '108390',
   'to': '631'},
  {'date': 'Jul 27, 2011',
   'fee': 'Loan fee:1,20 Mill. €',
   'from': '631',
   'player': '108390',
   'to': '13'},
  {'date': 'Jul 26, 2011',
   'fee': '8,95 Mill. €',
   'from': '1184',
   'player': '108390',
   'to': '631'},
  {'date': 'Jul 1, 2009',
   'fee': '-',
   'from': '3494',
   'player': '108390',
   'to': '1184'},
  {'date': 'Jul 1, 2008',
   'fee': '-',
   'from': '49393',
   'player': '108390',
   'to': '3494'},
  {'date': 'Jul 1, 2007',
   'fee': '-',
   'from': '34282',
   'player': '108390',
   'to': '49393'}])